In [0]:
#Understand the basic Image Classification pipeline and the data-driven
#approach (train/predict stages)
#● Data fetching and understand the train/val/test splits.
#● Implement and apply an optimal k-Nearest Neighbor (kNN) classifier (7.5
#points)
#● Print the classification metric report (2.5 points)
#● Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations) (5 points)
#● Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions) (2.5 points)
#● Implement batch normalization for training the neural network (2.5 points)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
images_of_numbers_file = h5py.File("/content/drive/My Drive/SVHN_single_grey1.h5", "r+")
list_of_keys = list(images_of_numbers_file.keys())
list_of_keys

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [0]:
images_of_numbers_file = h5py.File("/X_test = np.array(images_of_numbers_file['X_test'])
list_of_keys = list(images_of_numbers_file.keys())
list_of_keys

In [0]:
X_test = np.array(images_of_numbers_file['X_test'])
X_train = images_of_numbers_file['X_train']
X_val = images_of_numbers_file['X_val']
y_test = images_of_numbers_file['y_test']
y_train = images_of_numbers_file['y_train']
y_val = images_of_numbers_file['y_val']

In [0]:
# Converting data into arrays and then, normalizing image data from 0-255 to 0-1
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

In [30]:
# Data flatenning for KNN classifier
X_train_reshaped = np.reshape(X_train,(42000, 32*32))
X_test_reshaped = np.reshape(X_test,(18000, 32*32))
#Trying to find the right value of k:
list_opt = np.arange(1, 6)
accuracies = []
for Ks in list_opt:
 model_knn = KNeighborsClassifier(n_neighbors = Ks, algorithm = 'brute')
 model_knn = model_knn.fit(X_train_reshaped, y_train)

 # Evaluate the model and update the accuracies list
 y_pred = model_knn.predict(X_test_reshaped)
 acc_score = accuracy_score(y_test, y_pred)
 print(acc_score)
 accuracies.append(acc_score)

0.45916666666666667
0.25005555555555553
0.3783888888888889
0.24616666666666667
0.32416666666666666


In [10]:
i = int(np.argmax(accuracies))
print("k=", list_opt[i], "achieved highest accuracy of", accuracies[i] * 100, "on validation data")

k= 4 achieved highest accuracy of 48.13333333333333 on validation data


In [0]:
# Our final KNN model with k = 1
model_knn_2 = KNeighborsClassifier(n_neighbors = 47, algorithm = 'brute')
model_knn_2 = model_knn_2.fit(X_train_reshaped, y_train)
y_pred_2 = model_knn_2.predict(X_test_reshaped)


In [12]:
# The classification report is as follows:
class_report = classification_report(y_test, y_pred_2)
print(class_report)

              precision    recall  f1-score   support

           0       0.45      0.71      0.55      1814
           1       0.45      0.74      0.56      1828
           2       0.65      0.53      0.59      1803
           3       0.50      0.44      0.47      1719
           4       0.65      0.65      0.65      1812
           5       0.55      0.41      0.47      1768
           6       0.53      0.43      0.48      1832
           7       0.69      0.62      0.65      1808
           8       0.49      0.39      0.43      1812
           9       0.55      0.47      0.51      1804

    accuracy                           0.54     18000
   macro avg       0.55      0.54      0.54     18000
weighted avg       0.55      0.54      0.54     18000



In [0]:
accuracy = accuracy_score(y_test, y_pred_2)
c_matrix = confusion_matrix(y_test, y_pred_2)

In [0]:
# Converting labels to one hot vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)


In [15]:
# Deep neural network
model_dnn = Sequential()
#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn.add(Reshape((1024,),input_shape=(32,32,)))
# 1st hidden layer
model_dnn.add(Dense(200, activation='relu'))
# 2nd hidden layer
model_dnn.add(Dense(100, activation='relu'))
# Dropout layer
model_dnn.add(Dropout(0.25))
# 3rd hidden layer
model_dnn.add(Dense(60, activation='relu'))
# 4th hidden layer
model_dnn.add(Dense(30, activation='relu'))
# Output layer
model_dnn.add(Dense(10, activation='softmax'))
# Loss and Optimizer
model_dnn.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [17]:
# Training the model
model_dnn.fit(X_train, y_train, batch_size=32, nb_epoch=10, validation_data=(X_test, y_test))

Train on 42000 samples, validate on 18000 samples
Epoch 1/10
 1216/42000 [..............................] - ETA: 5s - loss: 0.8453 - acc: 0.7352

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


42000/42000 [==============================] - 6s 150us/step - loss: 0.7824 - acc: 0.7547 - val_loss: 0.7817 - val_acc: 0.7641
Epoch 2/10
42000/42000 [==============================] - 6s 149us/step - loss: 0.7744 - acc: 0.7580 - val_loss: 0.8257 - val_acc: 0.7505
Epoch 3/10
42000/42000 [==============================] - 6s 151us/step - loss: 0.7675 - acc: 0.7588 - val_loss: 0.7554 - val_acc: 0.7672
Epoch 4/10
42000/42000 [==============================] - 6s 149us/step - loss: 0.7588 - acc: 0.7624 - val_loss: 0.7213 - val_acc: 0.7828
Epoch 5/10
42000/42000 [==============================] - 6s 149us/step - loss: 0.7428 - acc: 0.7675 - val_loss: 0.7643 - val_acc: 0.7641
Epoch 6/10
42000/42000 [==============================] - 6s 149us/step - loss: 0.7344 - acc: 0.7702 - val_loss: 0.7752 - val_acc: 0.7587
Epoch 7/10
42000/42000 [==============================] - 6s 150us/step - loss: 0.7278 - acc: 0.7698 - val_loss: 0.7215 - val_acc: 0.7765
Epoch 8/10
42000/42000 [=====================

In [18]:
model_dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [19]:
# Clearing out tensorflow memory
clear_session()
# Define deep neural network Model
model_dnn_2 = Sequential()
#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn_2.add(Reshape((1024,),input_shape=(32,32,)))
# 1st hidden layer
model_dnn_2.add(Dense(200, activation='relu'))
# 2nd hidden layer
model_dnn_2.add(Dense(100, activation='relu'))
# Dropout layer
model_dnn_2.add(Dropout(0.25))
# 3rd hidden layer
model_dnn_2.add(Dense(60, activation='relu'))
# 4th hidden layer
model_dnn_2.add(Dense(30, activation='relu'))
# Output layer
model_dnn_2.add(Dense(10, activation='softmax'))
# Loss and Optimizer
sgd_optimizer = SGD(lr = 0.05)
model_dnn_2.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])


In [21]:
# Training the model
model_dnn_2.fit(X_train, y_train, batch_size=32, nb_epoch=10, validation_data=(X_test, y_test))

Train on 42000 samples, validate on 18000 samples
Epoch 1/10
  896/42000 [..............................] - ETA: 7s - loss: 2.2563 - acc: 0.1462

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


42000/42000 [==============================] - 6s 148us/step - loss: 2.3009 - acc: 0.1017 - val_loss: 2.3030 - val_acc: 0.1008
Epoch 2/10
42000/42000 [==============================] - 6s 145us/step - loss: 2.3031 - acc: 0.1003 - val_loss: 2.3031 - val_acc: 0.0982
Epoch 3/10
42000/42000 [==============================] - 8s 184us/step - loss: 2.3029 - acc: 0.1009 - val_loss: 2.3037 - val_acc: 0.0955
Epoch 4/10
42000/42000 [==============================] - 6s 150us/step - loss: 2.3030 - acc: 0.0993 - val_loss: 2.3026 - val_acc: 0.1007
Epoch 5/10
42000/42000 [==============================] - 6s 149us/step - loss: 2.3043 - acc: 0.0991 - val_loss: 2.3028 - val_acc: 0.0955
Epoch 6/10
42000/42000 [==============================] - 6s 147us/step - loss: 2.3030 - acc: 0.0976 - val_loss: 2.3031 - val_acc: 0.0955
Epoch 7/10
42000/42000 [==============================] - 6s 146us/step - loss: 2.3030 - acc: 0.0983 - val_loss: 2.3030 - val_acc: 0.1004
Epoch 8/10
42000/42000 [=====================

In [22]:
model_dnn_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [24]:
# Define deep neural network Model
model_dnn_3 = Sequential()
#Reshape data from 2D to 1D -> 32x32 to 1024
model_dnn_3.add(Reshape((1024,),input_shape=(32,32,)))
#Normalize the data
model_dnn_3.add(BatchNormalization())
# 1st hidden layer
model_dnn_3.add(Dense(200, activation='relu'))
model_dnn_3.add(BatchNormalization())
# 2nd hidden layer
model_dnn_3.add(Dense(100, activation='relu'))
model_dnn_3.add(BatchNormalization())
# Dropout layer
model_dnn_3.add(Dropout(0.25))
# 3rd hidden layer
model_dnn_3.add(Dense(60, activation='relu'))
model_dnn_3.add(BatchNormalization())
# 4th hidden layer
model_dnn_3.add(Dense(30, activation='relu'))
model_dnn_3.add(BatchNormalization())
# Output layer
model_dnn_3.add(Dense(10, activation='softmax'))
# Loss and Optimizer
model_dnn_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Training the model
model_dnn_3.fit(X_train, y_train, batch_size=32, nb_epoch=10, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 42000 samples, validate on 18000 samples
Epoch 1/10
42000/42000 [==============================] - 15s 360us/step - loss: 1.7073 - acc: 0.4193 - val_loss: 1.1083 - val_acc: 0.6627
Epoch 2/10
42000/42000 [==============================] - 14s 322us/step - loss: 1.2393 - acc: 0.6060 - val_loss: 0.9841 - val_acc: 0.6969
Epoch 3/10
42000/42000 [==============================] - 14s 322us/step - loss: 1.1154 - acc: 0.6471 - val_loss: 0.8580 - val_acc: 0.7339
Epoch 4/10
42000/42000 [==============================] - 14s 324us/step - loss: 1.0310 - acc: 0.6735 - val_loss: 0.7931 - val_acc: 0.7538
Epoch 5/10
42000/42000 [==============================] - 13s 320us/step - loss: 0.9635 - acc: 0.6952 - val_loss: 0.7744 - val_acc: 0.7597
Epoch 6/10
42000/42000 [==============================] - 13s 321us/step - loss: 0.9036 - acc: 0.7159 - val_loss: 0.6962 - val_acc: 0.7852
Epoch 7/10
42000/42000 [==============================] - 13s 320us/step - loss: 0.8740 - acc: 0.7252 - val_loss: 0.

In [0]:
# Prediction
y_pred_3 = model_dnn_3.predict(X_test)

In [0]:
# The classification report is as follows:
class_report = classification_report(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))

In [27]:
# Also, finding the accuracy and the confusion matrix to evaluate the model:
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))
c_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_3.argmax(axis=1))
print("The accuracy is", round(accuracy*100, 2), "% and the confusion matrix is\n", c_matrix)

The accuracy is 81.13 % and the confusion matrix is
 [[1566   35    6   22   28   16   39   28   34   40]
 [  32 1565   18   32   41   22   24   64   13   17]
 [  19   39 1425   52   25   19   19  137   25   43]
 [  19   44   31 1301   15  179   16   47   42   25]
 [  26   63   28   29 1524   11   52   11   20   48]
 [  14   32   16   85    9 1445   62   16   52   37]
 [  42   40   13   18   49   77 1428   13  138   14]
 [  16   61   33   21   15   11   17 1598   10   26]
 [  30   55   26   49   26   78  145   22 1324   57]
 [  68   56   22   37   31   66   21   36   39 1428]]


In [0]:
# The accuracy of the KNN model is coming out to be 51.33%, whereas, the accuracy of the deep neural network model is coming out to be
# 84.2%.
# Deep Neural network has better accuracy than KNN
# KNN doesn't work with large data.
# Huge training data is needed to get better results in NN
# CPU processing
# From the above classication metrics, we can see that KNN is denitely not a good choice for this image classication model and the use of
# DNN model is truly justied.


In [0]:
images_of_numbers_file.close()